# Implementación Modelo J-PAS #

In [810]:
#Paquetes necesarios (seguramente sobra alguno)
import numpy as np
import math
import pandas as pd
import scipy as sp
import sympy
import scipy.integrate as integrate
import scipy.special as special
from mpmath import *
import numpy.testing as testing
import camb

Ejemplo de función vectorizada

In [811]:
# Your scalar function
def my_function(x):
    # Example function: square the input
    return x**2

# Vectorize the function
vectorized_function = np.vectorize(my_function)

# Input array
input_array = np.array([1, 2, 3, 4, 5])

# Apply the vectorized function to the array
result_array = vectorized_function(input_array)

In [812]:
vectorized_function(input_array)

array([ 1,  4,  9, 16, 25])

# Modelo Kaiser + fotometría #

Bines de k y z

In [813]:
#Bines de k, con paso 0.025 en log:
KArrayCompleto = np.exp( np.arange(math.log(0.001), math.log(2.4900), 0.025) )
KArray = KArrayCompleto[range(121,245)]

In [814]:
#Bines de z:
za = np.arange(1.7, 3, 0.2)

Parámetros cosmlógicos

In [815]:
#Parámetros cosmológicos directos:
hJPAS = 0.674; c = 2.99792458E5; OmegabJPASh2 = 0.02212; OmegaCDMJPASh2 = 0.1206; 
OmegakJPAS = 0; OmegamJPAS = 0.314170; AsJPAS = 2.09052E-9; nsJPAS = 0.9626; 
sigma8JPAS = 0.8120;


In [816]:
#Parámetros cosmológicos indirectos:
HJPAS = 1/(c/100); OmegabJPAS = OmegabJPASh2/hJPAS**2; OmegaCDMJPAS = OmegaCDMJPASh2/hJPAS**2;
OmegaLJPAS = 1 - OmegamJPAS;

In [817]:
#Parámetros cosmológicos fuera del fiducial:
hJPASNoFid = hJPAS + hJPAS/100;
OmegabJPASh2NoFid = OmegabJPASh2 + OmegabJPASh2/100;
OmegaCDMJPASh2NoFid = OmegaCDMJPASh2 + OmegaCDMJPASh2/100; 
OmegamJPASNoFid = OmegamJPAS + OmegamJPAS/100;

#Parámetros cosmológicos indirectos fuera del fiducial:
OmegabJPASNoFid = OmegabJPASh2NoFid/hJPASNoFid**2; OmegaCDMJPASNoFid = OmegaCDMJPASh2NoFid/hJPASNoFid**2;
OmegaLJPASNoFid = 1 - OmegamJPASNoFid;

Parámetros cosmológicos directos fuera del fiducial

In [818]:
#Parámetros cosmológicos fuera del fiducial:
hJPASNoFid = hJPAS + hJPAS/100;
OmegabJPASh2NoFid = OmegabJPASh2 + OmegabJPASh2/100;
OmegaCDMJPASh2NoFid = OmegaCDMJPASh2 + OmegaCDMJPASh2/100;
OmegamJPASNoFid = (OmegaCDMJPASh2NoFid + OmegabJPASh2NoFid)/hJPAS**2

P(k) de materia desde CAMB

In [819]:
#Llamamos a CAMB aquí.
#Damos los parámetros cosmológicos para CAMB
pars = camb.CAMBparams()
pars.set_cosmology(H0=100*hJPAS, ombh2=OmegabJPASh2, omch2=OmegaCDMJPASh2, mnu=0, omk=0, tau=0.06);
pars.set_matter_power(redshifts=za, kmax=KArrayCompleto[-1]);

Note: redshifts have been re-sorted (earliest first)


In [820]:
#Aquí ya trabajamos con los resultados de CAMB 'results' y 'background':
results = camb.get_results(pars)
background = camb.get_background(pars, no_thermo=False)

#Con este comando calculamos el P de materia (en za) en el array de K.
#Se guardan los 3 arrays a la vez ya que el output los da así:
kmatCAMB, zaCAMB, PmatCAMB = results.get_matter_power_spectrum(minkh=KArrayCompleto[0], maxkh=KArrayCompleto[-1], npoints = len(KArrayCompleto))

In [821]:
#Importamos otro CAMB leyendo datos a z=0
parsz0 = camb.CAMBparams()
parsz0.set_cosmology(H0=100*hJPAS, ombh2=OmegabJPASh2, omch2=OmegaCDMJPASh2NoFid, mnu=0, omk=0, tau=0.06);
parsz0.set_matter_power(redshifts=[0], kmax=KArrayCompleto[-1]);
resultsz0 = camb.get_results(parsz0)
backgroundz0 = camb.get_background(parsz0, no_thermo=False)
kmatCAMBz0, zaCAMBz0, PmatCAMBz0 = resultsz0.get_matter_power_spectrum(minkh=KArrayCompleto[0], maxkh=KArrayCompleto[-1], npoints = len(KArrayCompleto))

In [822]:
#Importamos otro CAMB leyendo datos fuera del fiducial
parsNoFid = camb.CAMBparams()
parsNoFid.set_cosmology(H0=100*hJPASNoFid, ombh2=OmegabJPASh2, omch2=OmegaCDMJPASh2NoFid, mnu=0, omk=0, tau=0.06);
parsNoFid.set_matter_power(redshifts=[0], kmax=KArrayCompleto[-1]);
resultsz0 = camb.get_results(parsz0)
backgroundz0 = camb.get_background(parsz0, no_thermo=False)
kmatCAMBz0, zaCAMBz0, PmatCAMBz0 = resultsz0.get_matter_power_spectrum(minkh=KArrayCompleto[0], maxkh=KArrayCompleto[-1], npoints = len(KArrayCompleto))

In [823]:
#Otro CAMB leyendo zaupper
parszupper = camb.CAMBparams()
parszupper.set_cosmology(H0=100*hJPAS, ombh2=OmegabJPASh2, omch2=OmegaCDMJPASh2NoFid, mnu=0, omk=0, tau=0.06);
parszupper.set_matter_power(redshifts=za+0.1, kmax=KArrayCompleto[-1]);
resultszupper = camb.get_results(parszupper)
backgroundzupper = camb.get_background(parszupper, no_thermo=False)

Note: redshifts have been re-sorted (earliest first)


In [824]:
#Otro CAMB leyendo zalower
parszlower = camb.CAMBparams()
parszlower.set_cosmology(H0=100*hJPAS, ombh2=OmegabJPASh2, omch2=OmegaCDMJPASh2NoFid, mnu=0, omk=0, tau=0.06);
parszlower.set_matter_power(redshifts=za-0.1, kmax=KArrayCompleto[-1]);
resultszlower = camb.get_results(parszlower)
backgroundzlower = camb.get_background(parszlower, no_thermo=False)

Note: redshifts have been re-sorted (earliest first)


In [825]:
#Se interpola en k el P(k) a z=0:
def PmatInterCAMB(k):
  return np.interp(k, kmatCAMBz0, PmatCAMBz0[0])

Funciones E(z) y f(z) desde CAMB

In [826]:
#Leemos el valor que usa CAMB para Omegam (¿distinto de lo esperado?)
OmegamCAMB = np.array(results.get_Omega('baryon'))+np.array(results.get_Omega('cdm'))

In [827]:
EzCAMB = np.sqrt( OmegamCAMB*(1+za)**3+(1-OmegamCAMB) )

In [828]:
HCAMB = HJPAS * EzCAMB

In [829]:
#Este es el sigma8 para los bines de z
sigma8CAMBReves = np.array(results.get_sigma8())
sigma8CAMB = sigma8CAMBReves[::-1]

In [830]:
#Llamando a fsigma8 de CAMB y dividiendo entre sigma8 obtenemos f(z).
#La ordenación se hace de z más alto a z más bajo y la cambiamos
fCAMBReves = np.array(results.get_fsigma8())/sigma8CAMBReves
fCAMB = fCAMBReves[::-1]

In [831]:
#El growth factor D(z) no está directamente en CAMB, pero su forma es igual que la de sigmaR.
#Por tanto, usamos el sigmaR normalizado
DeCAMBReves = results.get_sigmaR(8)/resultsz0.get_sigmaR(8)[0]
DeCAMB = DeCAMBReves[::-1]

Función A(z)

In [832]:
def bJPAS(z):
  return 0.53+0.289*(1+z)**2

In [833]:
#Notar aquí que el sigma8JPAS se pone por seguir la notación de Maroto, ya que luego se va
ACAMB = DeCAMB*bJPAS(za)*sigma8JPAS

Función R(z)

In [834]:
RCAMB = DeCAMB*fCAMB*sigma8JPAS

Función sigmar(z)

In [835]:
DeltazJPAS = 0.00364236313918151

In [836]:
sigmarCAMB = DeltazJPAS*(1+za)/HCAMB

In [837]:
def PmatInterCAMBz0(k):
    return PmatInterCAMB(k)/(DeCAMB[0]**2)

Modelo Kaiser

In [838]:
def PJPASAntesFingersAntesAPCAMB(mu,k):
  return (ACAMB[0]+RCAMB[0]*mu**2)**2*PmatInterCAMB(k)/sigma8JPAS**2*np.exp(-(k*mu*sigmarCAMB[0])**2)

In [839]:
PJPASAntesFingersAntesAPCAMB(0.1,0.2)

2727.9330960696807

# Fingers of God #

Parámetro dispersión velocidades. Parece que esto hay que hacerlo solo 1 vez, con el Pmat del fiducial

In [840]:
import scipy.integrate as integrate

In [841]:
sigmapJPAS = (1/(6*np.pi**2)*(DeCAMB/1)**2*integrate.quad(lambda k: PmatInterCAMB(k), KArrayCompleto[0], KArrayCompleto[len(KArrayCompleto)-1])[0])**0.5

/var/folders/1h/0trg724n2fg2pn5j02gypgzc0000gn/T/ipykernel_40184/3390978684.py:1: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  sigmapJPAS = (1/(6*np.pi**2)*(DeCAMB/1)**2*integrate.quad(lambda k: PmatInterCAMB(k), KArrayCompleto[0], KArrayCompleto[len(KArrayCompleto)-1])[0])**0.5


Cómputo factor Fingers of God

In [842]:
def FFog(mu,k):
    return 1/(1+(fCAMB[0]*k*mu*sigmapJPAS[0])**2)

P(k) de galaxias con FFoG

In [843]:
def PJPASAntesAPCAMB(mu,k):
    return FFog(mu,k)*PJPASAntesFingersAntesAPCAMB(mu,k)

# Efecto AP #

Integración de la distancia comóvil

In [844]:
XiCAMB = results.comoving_radial_distance(za)*hJPAS

Distancia angular

In [845]:
#Multiplicamos por h para tener unidades h/Mpc:
DaCAMB = XiCAMB/(1+za)

Función de distorsión del AP

In [846]:
def QCAMB(mu):
    return ((EzCAMB[0]*XiCAMB[0]*mu**2-EzCAMB[0]**2*XiCAMB[0]**2*(mu**2-1))**0.5/(EzCAMB[0]*XiCAMB[0]))

Mus y ks distorsionados

In [847]:
def muNoFidCAMB(mu):
    return mu*  EzCAMB[0]/(EzCAMB[0]*QCAMB(mu))

In [848]:
def kNoFidCAMB(mu,k):
    return QCAMB(mu)*k

Factor AP a incluir en el P de galaxias:

In [849]:
FactorAPCAMB = DaCAMB**2*EzCAMB/( DaCAMB**2*EzCAMB )

P de galaxias con factor AP y los anteriores

In [850]:
def PgCAMB(mu,k):
    return FactorAPCAMB[0]*FFog(muNoFidCAMB(mu),kNoFidCAMB(mu,k))*(ACAMB[0]+RCAMB[0]*muNoFidCAMB(mu)**2)**2 * PmatInterCAMB(kNoFidCAMB(mu,k))/sigma8JPAS**2 *np.exp(-(k*mu*sigmarCAMB[0])**2)

# P galaxy monopole #

In [851]:
def PgmonopoleCAMB(k, axis = 0):
    mu = np.arange(-0.90, 0.90, 1/100)
    return 1/2 * integrate.trapz(mu, (lambda x: PgCAMB(x, k))(mu), axis = axis)

In [852]:
PgmonopoleCAMBValores = np.zeros(len(KArray)-1)
for i in range(0, len(KArray)-1):
    PgmonopoleCAMBValores[i] = PgmonopoleCAMB(KArray[i])

In [853]:
PgmonopoleCAMBValores

array([ 1.06010749e+05,  1.07365348e+05,  1.08690847e+05,  1.09983893e+05,
        1.11240964e+05,  1.12458356e+05,  1.13632208e+05,  1.14758513e+05,
        1.15833092e+05,  1.16851586e+05,  1.17809500e+05,  1.18702209e+05,
        1.19524921e+05,  1.20272684e+05,  1.20940446e+05,  1.21523082e+05,
        1.22015362e+05,  1.22411961e+05,  1.22707558e+05,  1.22896862e+05,
        1.22974599e+05,  1.22935540e+05,  1.22774609e+05,  1.22486924e+05,
        1.22067759e+05,  1.21512601e+05,  1.20817272e+05,  1.19978052e+05,
        1.18991804e+05,  1.17856009e+05,  1.16568791e+05,  1.15128907e+05,
        1.13535622e+05,  1.11788795e+05,  1.09888975e+05,  1.07837822e+05,
        1.05638486e+05,  1.03295501e+05,  1.00814498e+05,  9.82021040e+04,
        9.54658616e+04,  9.26143295e+04,  8.96571827e+04,  8.66053929e+04,
        8.34710145e+04,  8.02671829e+04,  7.70077552e+04,  7.37073028e+04,
        7.03809646e+04,  6.70444084e+04,  6.37131828e+04,  6.04025218e+04,
        5.71271577e+04,  

# Covarianza #

Importación y lectura de las densidades

In [854]:
ImportacionDensityHighZ = [i.strip().split() for i in open("/Users/guillermo/Desktop/DensityHighZ.dat").readlines()]

In [855]:
DensityHighZ = np.zeros(len(ImportacionDensityHighZ));

In [856]:
for i in range(0, len(ImportacionDensityHighZ)):
  DensityHighZ[i] = ImportacionDensityHighZ[i][1]

Definición de funciones necesarias para la covarianza.
Definición del volumen (requiere distancia angular con unidades), y binear en zupper y zlower

In [857]:
XiConC = c*XiCAMB

In [858]:
zaupper = za+(za[[1]]-za[[0]])/2;    zalower = za-(za[[1]]-za[[0]])/2;

In [859]:
XiCAMBZaLower = resultszlower.comoving_radial_distance(zalower)*hJPAS
XiCAMBZaUpper = resultszupper.comoving_radial_distance(zaupper)*hJPAS

In [860]:
fsky = 0.2575;

In [861]:
Vol = 4*np.pi*fsky/3*(XiCAMBZaUpper**3-XiCAMBZaLower**3)

Definición del número de modos (requiere arrays en kupper y klower)

In [862]:
KArrayUpper = np.zeros(len(KArray)); KArrayLower = np.zeros(len(KArray));

In [863]:
for i in range(0, len(KArray)-1):
  KArrayUpper[i] = KArray[i] + (KArray[i+1]-KArray[i])/2;   KArrayLower[i] = KArray[i] - (KArray[i+1]-KArray[i])/2;

In [864]:
KArrayUpper[len(KArray)-1] = KArrayUpper[len(KArray)-2];  KArrayLower[len(KArray)-1] = KArrayLower[len(KArray)-2];

In [865]:
def Nk(k1,k2):
    return Vol[0] * (4*np.pi/3*(k1**3-k2**3))/((2*np.pi)**3)

In [866]:
NkEvaluado = np.zeros(len(KArray)-1)
for i in range(0, len(KArray)-1):
    NkEvaluado[i] = Nk(KArrayUpper[i],KArrayLower[i])

Definición de la covarianza

In [867]:
def Cov(k,k1,k2):
    return 2 * (PgmonopoleCAMB(k) + 1/DensityHighZ[0])**2 / Nk(k1,k2)

In [868]:
Cov = (PgmonopoleCAMBValores + 1/DensityHighZ[0])**2 / NkEvaluado

# Lectura de datos #

Importación del Monopolo simulado

In [869]:
ImportacionDatosHighZ = [i.strip().split() for i in open("/Users/guillermo/Desktop/FicticioHighZArrayEnK.dat").readlines()]

In [870]:
DataPkzTotal = np.zeros(len(za));

In [871]:
Pk1 = np.zeros(len(ImportacionDatosHighZ)); Pk2 = np.zeros(len(ImportacionDatosHighZ));
Pk3 = np.zeros(len(ImportacionDatosHighZ)); Pk4 = np.zeros(len(ImportacionDatosHighZ));
Pk5 = np.zeros(len(ImportacionDatosHighZ)); Pk6 = np.zeros(len(ImportacionDatosHighZ));
Pk7 = np.zeros(len(ImportacionDatosHighZ));

In [872]:
for i in range(0, len(ImportacionDatosHighZ)-1):
  Pk1[i] = ImportacionDatosHighZ[i][2]; Pk2[i] = ImportacionDatosHighZ[i][3]; Pk3[i] = ImportacionDatosHighZ[i][4];
  Pk4[i] = ImportacionDatosHighZ[i][5]; Pk5[i] = ImportacionDatosHighZ[i][6]; Pk6[i] = ImportacionDatosHighZ[i][7];
  Pk7[i] = ImportacionDatosHighZ[i][8]

In [873]:
DataPkzTotal = np.array([Pk1,Pk2,Pk3,Pk4,Pk5,Pk6,Pk7])

# Likelihood #

Bineamos los elementos del likelihood:

In [874]:
CovBineadoz1 = np.zeros(len(KArray)); CovBineadoz2 = np.zeros(len(KArray));
CovBineadoz3 = np.zeros(len(KArray)); CovBineadoz4 = np.zeros(len(KArray));
CovBineadoz5 = np.zeros(len(KArray)); CovBineadoz6 = np.zeros(len(KArray));
CovBineadoz7 = np.zeros(len(KArray));

In [875]:
CovBineado = np.array([CovBineadoz1,CovBineadoz2,CovBineadoz3,CovBineadoz4,CovBineadoz5,CovBineadoz6,CovBineadoz7])

In [876]:
for i in range(0, len(KArray)-1):
    CovBineado[0][i] = Cov[i]

In [877]:
PgBineadoz1 = np.zeros(len(KArray)); PgBineadoz2 = np.zeros(len(KArray));
PgBineadoz3 = np.zeros(len(KArray)); PgBineadoz4 = np.zeros(len(KArray));
PgBineadoz5 = np.zeros(len(KArray)); PgBineadoz6 = np.zeros(len(KArray));
PgBineadoz7 = np.zeros(len(KArray));

In [878]:
PgBineado = np.array([PgBineadoz1,PgBineadoz2,PgBineadoz3,PgBineadoz4,PgBineadoz5,PgBineadoz6,PgBineadoz7])

In [879]:
for i in range(0, len(KArray)-1):
    PgBineado[0][i] = PgmonopoleCAMBValores[i]

In [880]:
def lnlike(j,i):
    return (PgBineado[j][i]-DataPkzTotal[j][i])**2 * 1/CovBineado[j][i] + np.log(CovBineado[j][i])

In [881]:
arrayindices = np.arange(0,len(KArray)-50,1)
lnlike(0,arrayindices)
sumlnlike = np.sum(lnlike(0,arrayindices))

In [882]:
sumlnlike

48118.66360607402